In [1]:
from trt_inferer import TRTInference
import tensorrt as trt
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.set_grad_enabled(False)
print(f"using torch {torch.__version__}")
print(f"using pyTensorRT {trt.__version__}")

using pyTensorRT 8.5.1.7
using torch 1.12.1+cu116
using pyTensorRT 8.5.1.7


# DUMMY model

In [2]:
class Clamp_fp16_bug(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.AvgPool2d(kernel_size=(3,3))
    
    def forward(self, x):
        x = self.pool(x)# + 500
        x = torch.clamp(x, min=0., max=1000.)
        return x

In [3]:
inputs_t = torch.randn(1,3,400,400).cuda()
inputs_np = inputs_t.cpu().numpy()

In [4]:
model = Clamp_fp16_bug().cuda()

In [5]:
outputs = model(inputs_t)
outputs.max()

tensor(1.5061, device='cuda:0')

# Export to ONNX

In [6]:
torch.onnx.export(model.half(),
                inputs_t.half(),
                "bug_fp16.onnx",
                input_names=["inputs"],
                output_names=["outputs"],
                export_params=True,
                verbose=False,
                opset_version=16,
                do_constant_folding=True,
                training=torch.onnx.TrainingMode.EVAL,
                )

In [7]:
!/usr/src/tensorrt/bin/trtexec --onnx=bug_fp16.onnx --saveEngine=bug_fp16.engine --fp16

&&&& RUNNING TensorRT.trtexec [TensorRT v8501] # /usr/src/tensorrt/bin/trtexec --onnx=bug_fp16.onnx --saveEngine=bug_fp16.engine --fp16
[12/04/2022-16:28:33] [I] === Model Options ===
[12/04/2022-16:28:33] [I] Format: ONNX
[12/04/2022-16:28:33] [I] Model: bug_fp16.onnx
[12/04/2022-16:28:33] [I] Output:
[12/04/2022-16:28:33] [I] === Build Options ===
[12/04/2022-16:28:33] [I] Max batch: explicit batch
[12/04/2022-16:28:33] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[12/04/2022-16:28:33] [I] minTiming: 1
[12/04/2022-16:28:33] [I] avgTiming: 8
[12/04/2022-16:28:33] [I] Precision: FP32+FP16
[12/04/2022-16:28:33] [I] LayerPrecisions: 
[12/04/2022-16:28:33] [I] Calibration: 
[12/04/2022-16:28:33] [I] Refit: Disabled
[12/04/2022-16:28:33] [I] Sparsity: Disabled
[12/04/2022-16:28:33] [I] Safe mode: Disabled
[12/04/2022-16:28:33] [I] DirectIO mode: Disabled
[12/04/2022-16:28:33] [I] Restricted mode: Disabled
[12/04/2022-16:28:33] [I] Bu

[12/04/2022-16:28:48] [I] 
[12/04/2022-16:28:48] [I] === Performance summary ===
[12/04/2022-16:28:48] [I] Throughput: 8681.96 qps
[12/04/2022-16:28:48] [I] Latency: min = 0.092041 ms, max = 1.20581 ms, mean = 0.127789 ms, median = 0.129883 ms, percentile(90%) = 0.142822 ms, percentile(95%) = 0.147461 ms, percentile(99%) = 0.159851 ms
[12/04/2022-16:28:48] [I] Enqueue Time: min = 0.00436401 ms, max = 0.0485229 ms, mean = 0.0112367 ms, median = 0.0102539 ms, percentile(90%) = 0.015625 ms, percentile(95%) = 0.0168457 ms, percentile(99%) = 0.0246582 ms
[12/04/2022-16:28:48] [I] H2D Latency: min = 0.0742798 ms, max = 1.15234 ms, mean = 0.0892251 ms, median = 0.0883789 ms, percentile(90%) = 0.0957031 ms, percentile(95%) = 0.0982666 ms, percentile(99%) = 0.111816 ms
[12/04/2022-16:28:48] [I] GPU Compute Time: min = 0.00683594 ms, max = 0.0449219 ms, mean = 0.0136768 ms, median = 0.0133057 ms, percentile(90%) = 0.0205078 ms, percentile(95%) = 0.0214844 ms, percentile(99%) = 0.0256348 ms
[12/0

In [8]:
torch.onnx.export(model,
                inputs_t,
                "bug_fp32.onnx",
                input_names=["inputs"],
                output_names=["outputs"],
                export_params=True,
                verbose=False,
                opset_version=16,
                do_constant_folding=True,
                training=torch.onnx.TrainingMode.EVAL,
                )

In [9]:
!/usr/src/tensorrt/bin/trtexec --onnx=bug_fp32.onnx --saveEngine=bug_fp32.engine --fp16

&&&& RUNNING TensorRT.trtexec [TensorRT v8501] # /usr/src/tensorrt/bin/trtexec --onnx=bug_fp32.onnx --saveEngine=bug_fp32.engine --fp16
[12/04/2022-16:28:48] [I] === Model Options ===
[12/04/2022-16:28:48] [I] Format: ONNX
[12/04/2022-16:28:48] [I] Model: bug_fp32.onnx
[12/04/2022-16:28:48] [I] Output:
[12/04/2022-16:28:48] [I] === Build Options ===
[12/04/2022-16:28:48] [I] Max batch: explicit batch
[12/04/2022-16:28:48] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[12/04/2022-16:28:48] [I] minTiming: 1
[12/04/2022-16:28:48] [I] avgTiming: 8
[12/04/2022-16:28:48] [I] Precision: FP32+FP16
[12/04/2022-16:28:48] [I] LayerPrecisions: 
[12/04/2022-16:28:48] [I] Calibration: 
[12/04/2022-16:28:48] [I] Refit: Disabled
[12/04/2022-16:28:48] [I] Sparsity: Disabled
[12/04/2022-16:28:48] [I] Safe mode: Disabled
[12/04/2022-16:28:48] [I] DirectIO mode: Disabled
[12/04/2022-16:28:48] [I] Restricted mode: Disabled
[12/04/2022-16:28:48] [I] Bu

[12/04/2022-16:29:03] [I] 
[12/04/2022-16:29:03] [I] === Performance summary ===
[12/04/2022-16:29:03] [I] Throughput: 5688.41 qps
[12/04/2022-16:29:03] [I] Latency: min = 0.173828 ms, max = 1.35254 ms, mean = 0.229397 ms, median = 0.229492 ms, percentile(90%) = 0.241333 ms, percentile(95%) = 0.245361 ms, percentile(99%) = 0.256439 ms
[12/04/2022-16:29:03] [I] Enqueue Time: min = 0.0065918 ms, max = 0.0541992 ms, mean = 0.013468 ms, median = 0.0131836 ms, percentile(90%) = 0.0157471 ms, percentile(95%) = 0.0176392 ms, percentile(99%) = 0.0307617 ms
[12/04/2022-16:29:03] [I] H2D Latency: min = 0.146362 ms, max = 1.27075 ms, mean = 0.172893 ms, median = 0.172058 ms, percentile(90%) = 0.18219 ms, percentile(95%) = 0.184814 ms, percentile(99%) = 0.198608 ms
[12/04/2022-16:29:03] [I] GPU Compute Time: min = 0.0078125 ms, max = 0.0480957 ms, mean = 0.0176824 ms, median = 0.017334 ms, percentile(90%) = 0.0205078 ms, percentile(95%) = 0.022522 ms, percentile(99%) = 0.0266113 ms
[12/04/2022-16:

# Check output via pyTensorRT

In [10]:
from trt_inferer import TRTInference
import tensorrt as trt

In [11]:
trt_model_path = "bug_fp16.engine"

inferer = TRTInference(trt_model_path)

trt_outp = inferer.infer(inputs_np.astype(inferer.input_dtype))
trt_outp#.max()


inputs shape: (1, 3, 400, 400)
outputs shape: (1, 3, 133, 133)
input_dtype <class 'numpy.float16'>
output_dtype <class 'numpy.float16'>
[12/04/2022-16:29:04] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars


array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]], dtype=float16)

In [12]:
trt_model_path = "bug_fp32.engine"

fp_32_inferer = TRTInference(trt_model_path)


trt_outp = fp_32_inferer.infer(inputs_np.astype(inferer.input_dtype))
trt_outp#.max()


inputs shape: (1, 3, 400, 400)
outputs shape: (1, 3, 133, 133)
input_dtype <class 'numpy.float32'>
output_dtype <class 'numpy.float32'>
[12/04/2022-16:29:05] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars


array([[[[5.47318816e-01, 3.70679647e-02, 0.00000000e+00, ...,
          0.00000000e+00, 1.07814625e-01, 6.29381463e-03],
         [2.72996002e-03, 4.26550269e-01, 0.00000000e+00, ...,
          0.00000000e+00, 7.01967394e-03, 2.99796835e-02],
         [0.00000000e+00, 0.00000000e+00, 2.22142227e-02, ...,
          4.38107967e-01, 1.01853050e-01, 0.00000000e+00],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          8.29727650e-02, 0.00000000e+00, 2.41145536e-01],
         [0.00000000e+00, 1.14657097e+01, 1.99925508e-02, ...,
          4.15930837e-01, 0.00000000e+00, 4.06617761e-01],
         [6.62461040e-04, 1.06577920e-02, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 3.47552204e+00]],

        [[2.64474470e-02, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 2.92849392e-01],
         [8.09557152e+00, 0.00000000e+00, 0.00000000e+00, ...,
          1.30735978e-03, 0.00000000e+00, 0.00000000e+00],
        

# ONNX runtime

In [13]:
import onnx
onnx_p = "bug_fp16.onnx"
onnx_model = onnx.load(onnx_p)
onnx.checker.check_model(onnx_model)

In [14]:
import onnxruntime as ort
import numpy as np

ort_sess = ort.InferenceSession(onnx_p)
outputs = ort_sess.run(None, {'inputs': inputs_np.astype(np.float16)})


outputs[0]

array([[[[3.8269e-02, 0.0000e+00, 3.0078e-01, ..., 2.4133e-01,
          2.0813e-01, 0.0000e+00],
         [9.7717e-02, 6.3916e-01, 3.3423e-01, ..., 4.0698e-01,
          1.7358e-01, 0.0000e+00],
         [7.6758e-01, 0.0000e+00, 3.7134e-01, ..., 1.9128e-01,
          0.0000e+00, 2.5732e-01],
         ...,
         [9.8206e-02, 2.9565e-01, 3.5004e-02, ..., 2.0972e-01,
          6.5723e-01, 9.0430e-01],
         [5.1172e-01, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00,
          1.7834e-01, 0.0000e+00],
         [1.0431e-01, 3.3911e-01, 0.0000e+00, ..., 0.0000e+00,
          2.6587e-01, 0.0000e+00]],

        [[0.0000e+00, 2.3083e-01, 3.9062e-02, ..., 0.0000e+00,
          0.0000e+00, 2.6611e-01],
         [4.5288e-02, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00,
          2.9907e-01, 0.0000e+00],
         [2.1753e-01, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00,
          2.5879e-01, 0.0000e+00],
         ...,
         [2.4255e-01, 0.0000e+00, 2.1152e-03, ..., 0.0000e+00,
          1.4197e-01, 1.0